# This notebook summarize additional features we can extract from images
1. Exposure = sum(any indicator of exposure: people, house, building, sidewalk)/total_num_image
2. Motorcycle = count()
3. Sidewalk Coverage = sum(indicator of sidewalk)/total_num_image
4. Crosswalk Coverage (later)
5. Bikelane Coverage (later)

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from sklearn.preprocessing import StandardScaler
import h3
from tqdm import tqdm


ROOT = "/lustre1/g/geog_pyloo/05_timemachine"
CURATED_FOLDER = f"{ROOT}/_curated"
EXPORT_FOLDER = f"{ROOT}/_curated/c_analysis"

IMG_SAMPLE_FOLDER = "/lustre1/g/geog_pyloo/05_timemachine/_curated/c_analysis/img_sample"

PANO_PATH = "{ROOT}/GSV/gsv_rgb/{cityabbr}/gsvmeta/gsv_pano.csv"
CURATED_TARGET = "/lustre1/g/geog_pyloo/05_timemachine/_curated/c_seg_hex"
META_PATH = "{ROOT}/GSV/gsv_rgb/{cityabbr}/gsvmeta/{cityabbr}_meta.csv"
OBJECT_DET = os.path.join(CURATED_FOLDER, "c_object_longitudinal")

N = 8
H3_RES = [9]
res = H3_RES[0]

In [ ]:

def get_std(df_seg_update, variables_remain):
    scaler = StandardScaler().fit(df_seg_update[variables_remain])
    data = scaler.transform(df_seg_update[variables_remain])
    return data

def load_class():
    serviceaccount = "/home/yuanzf/uvi-time-machine/google_drive_personal.json"
    import gspread

    # from oauth2client.service_account import ServiceAccountCredentials
    gc = gspread.service_account(filename=serviceaccount)

    def read_url(url, SHEET_NAME):
        SHEET_ID = url.split("/")[5]
        spreadsheet = gc.open_by_key(SHEET_ID)
        worksheet = spreadsheet.worksheet(SHEET_NAME)
        rows = worksheet.get_all_records()
        df_spread = pd.DataFrame(rows)
        return df_spread, worksheet

    url = "https://docs.google.com/spreadsheets/d/1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw/edit?usp=sharing"
    SHEETNAME = "object150"
    obj_meta, other_worksheet = read_url(url, SHEETNAME)
    return obj_meta

def get_result(cityabbr, curated_folder, f_suffixes = "*panoptic.csv"):
    outfolder = f"{curated_folder}/{cityabbr}"
    seg_file = glob(os.path.join(outfolder, f_suffixes))
    panoptic_df = []
    for p in seg_file:
        temp = pd.read_csv(p)
        panoptic_df.append(temp)
    panoptic_df = pd.concat(panoptic_df).reset_index(drop = True)
    return panoptic_df

def clean_seg(seg_df, pano_df, meta_df):

    seg_df_filtered = seg_df.merge(meta_df, on = 'img')
    seg_df_filtered = seg_df_filtered[seg_df_filtered['size']>=10000].reset_index(drop = True)
    print("Segmentation shape after filter: ", seg_df_filtered.shape[0])
    # count number of unique labels per image and drop it if image has fewer than 3
    seg_df_filtered['label_c'] = seg_df_filtered.groupby('img')['labels'].transform('nunique')
    seg_df_filtered = seg_df_filtered[seg_df_filtered['label_c']>=3].reset_index(drop = True)
    print("Segmentation shape after filter images with problems: ", seg_df_filtered.shape[0])
    
    seg_df_summary = seg_df_filtered.groupby(["img", "labels"]).agg({'areas':'sum'}).reset_index()
    seg_df_summary['panoid'] = seg_df_summary['img'].apply(lambda x: x[:22])

    col_cols = ["labels"]
    index_cols = ["img", "year", "h3_9"]
    seg_df_summary_pano = seg_df_summary.merge(pano_df, on = ['panoid'])
    
    
    if seg_df_summary_pano.shape[0]<seg_df_summary.shape[0]:
        print("data missing after data join.")
        print("Before join: ", seg_df_summary.shape[0])
        print("After join: ",seg_df_summary_pano.shape[0])
    else:
        print("data consistent")
    
    seg_df_summary = seg_df_summary_pano.drop_duplicates(index_cols+col_cols)
    print("Segmentation shape: ", seg_df_summary.shape[0])
    seg_df_pivot = seg_df_summary.pivot(
        columns = col_cols,
        index = index_cols,
        values = "areas"
    ).reset_index().fillna(0)
    return seg_df_pivot
def load_raw(city):
    cityabbr = city.lower().replace(" ", "")
    curate_folder = CURATED_FOLDER.format(ROOTFOLDER = ROOT)

    seg_df = get_result(cityabbr, CURATED_FOLDER, f_suffixes = "*seg.csv")
    
    pano_df = pd.read_csv(PANO_PATH.format(
    ROOT = ROOT,
    cityabbr = cityabbr
    ))[['panoid', 'lat', 'lon', 'year', 'month']]

    for res in [9]:
        pano_df[f'h3_{res}'] = pano_df.apply(lambda x: h3.geo_to_h3(x.lat, x.lon, res), axis=1)
        
    meta_df = pd.read_csv(META_PATH.format(
        ROOT = ROOT,
        cityabbr = cityabbr
    ))
    meta_df['img']= meta_df['path'].apply(lambda x: x.split("/")[-1].split(".")[0])
    # here make sure 
    meta_df = meta_df[['img','size','path']]
    return seg_df, meta_df, pano_df

def get_seg_data(city, hex_detail_cluster):
    cityabbr = city.lower().replace(" ", "")
    seg_df, meta_df, pano_df = load_raw(city)
    seg_df_pivot = clean_seg(seg_df, pano_df, meta_df)
    print(f"city {cityabbr} saved")
    print("*"*50)

    # hex_detail_cluster_city_sel = hex_detail_cluster[hex_detail_cluster['city_lower']==cityabbr].copy()
    # seg_df_pivot_sel = seg_df_pivot.merge(hex_detail_cluster_city_sel[['hex_id', 'cluster_8']], 
    #                                       left_on = 'h3_9', right_on = 'hex_id')\
    #     .drop('hex_id', axis = 1)
    seg_df_pivot_sel = seg_df_pivot.copy()
        
    new_cols = []
    for x in seg_df_pivot_sel.columns:
        if str(x) in obj_meta["id"].values:
            new_cols.append(ADE_CATEGORIES_DICT[str(x)])
        else:
            new_cols.append(str(x))
    seg_df_pivot_sel.columns = new_cols

    # drop the columns if all value are 0
    variables = set([v for v in seg_df_pivot_sel.columns if v in obj_meta["category"].unique()])
    print("Variables original: ", len(variables))
    to_drop = ["other"]
    variables_remain = [v for v in variables if not v in to_drop]
    print("Variables kept: ", len(variables_remain))
    seg_df_pivot_sel_stack = seg_df_pivot_sel.set_index(["img", "year", f"h3_{res}","cluster_8"]).stack().reset_index().fillna(0)\
        .groupby(["img", "year", f"h3_{res}","cluster_8","level_4"]).sum().reset_index()\
            .pivot_table(index = ["img", "year", f"h3_{res}","cluster_8"], columns = "level_4", values = 0).reset_index().fillna(0)\
                .drop(columns = ['other'], axis = 1)
    # standardize the data
    return seg_df_pivot_sel_stack, meta_df

def extract_seg(seg_df_pivot_sel):
    new_cols = []
    for x in seg_df_pivot_sel.columns:
        if str(x) in obj_meta["id"].values:
            new_cols.append(ADE_CATEGORIES_DICT[str(x)])
        else:
            new_cols.append(str(x))
    seg_df_pivot_sel.columns = new_cols

    # drop the columns if all value are 0
    variables = set([v for v in seg_df_pivot_sel.columns if v in obj_meta["category"].unique()])
    print("Variables original: ", len(variables))
    to_drop = ["other"]
    variables_remain = [v for v in variables if not v in to_drop]
    print("Variables kept: ", len(variables_remain))
    seg_df_pivot_sel_stack = seg_df_pivot_sel.set_index(["img", "year", f"h3_{res}"]).stack().reset_index().fillna(0)\
        .groupby(["img", "year", f"h3_{res}","level_3"]).sum().reset_index()\
            .pivot_table(index = ["img", "year", "h3_9"], columns = "level_3", values = 0).reset_index().fillna(0)\
                .drop(columns = ['other'], axis = 1)
    seg_df_pivot_sel_stack['sidewalk_indicator'] = seg_df_pivot_sel_stack['sidewalk'] > 0
    seg_df_pivot_sel_stack['exposure_indicator'] = seg_df_pivot_sel_stack[['building','house','table+chair','wall', 'window','person','bike','motorcycle','car']].sum(axis = 1) > 0
    seg_df_pivot_sel_stack['panoid'] = seg_df_pivot_sel_stack['img'].apply(lambda x: x[:22])
    seg_df_summary = seg_df_pivot_sel_stack\
        .groupby([f'h3_{res}'])\
            .agg({'sidewalk_indicator':'sum',
                'exposure_indicator':'sum',
                'img':'count',
                'panoid':'nunique'})\
                .reset_index()
    seg_df_summary['sidewalk_presence'] = seg_df_summary['sidewalk_indicator'] / seg_df_summary['img']
    seg_df_summary['exposure_presence'] = seg_df_summary['exposure_indicator'] / seg_df_summary['img']
    return seg_df_summary

def get_object_result(cityabbr, seg_df_summary):
    object_detect = pd.read_csv(f"{OBJECT_DET}/c_object_res={res}_{cityabbr}.csv")
    # calculate the total number of panoid per hex_id across all years
    object_detect_sum = object_detect.drop_duplicates(['year', 'hex_id','panoid_n']).groupby(['hex_id'])\
        .agg({'panoid_n':'sum'}).reset_index()
    object_detect_group = object_detect.groupby(['hex_id','object_name']).agg({
        'n':'sum'}).reset_index()\
            .merge(object_detect_sum, on = 'hex_id', how = 'left')
    object_detect_group['n_per_pano'] = object_detect_group['n'] / object_detect_group['panoid_n']
    object_detect_group_wide = object_detect_group[object_detect_group['object_name']!='0'].pivot(
        columns = 'object_name',
        index = 'hex_id',
        values = 'n_per_pano'
    ).fillna(0)
    object_detect_group_wide.columns = ['obj_'+str(x) for x in object_detect_group_wide.columns]
    seg_df_summary_update = seg_df_summary.merge(object_detect_group_wide, 
                                                left_on = 'h3_9', 
                                                right_index = True, 
                                                how = 'left')
    seg_df_summary_update.to_csv(f"{EXPORT_FOLDER}/{cityabbr}_seg_summary.csv", index = False)
    print(f"city {cityabbr} saved")
    return seg_df_summary_update

In [55]:
obj_meta = load_class()
obj_meta["id"] = obj_meta["id"].astype(str)
ADE_CATEGORIES_DICT = dict(zip(obj_meta["id"].values, obj_meta["category"].values))

In [ ]:
city = "New York"
cityabbr = city.lower().replace(" ", "")
seg_df, meta_df, pano_df = load_raw(city)
seg_df_pivot = clean_seg(seg_df, pano_df, meta_df)
seg_summary = extract_seg(seg_df_pivot)
seg_summary_update = get_object_result(cityabbr, seg_summary)

Segmentation shape after filter:  7943563
Segmentation shape after filter images with problems:  7942738
data consistent
Segmentation shape:  7942738


In [58]:
# loop through all cities
city_ls = pd.read_csv("../city_meta.csv")["City"].unique().tolist()
for city in tqdm(city_ls):
    if city == "New York":
        continue
    print(f"Processing {city}")
    try:
        cityabbr = city.lower().replace(" ", "")
        seg_df, meta_df, pano_df = load_raw(city)
        seg_df_pivot = clean_seg(seg_df, pano_df, meta_df)
        seg_summary = extract_seg(seg_df_pivot)
        seg_summary_update = get_object_result(cityabbr, seg_summary)
    except Exception as e:
        print(city + " failed")
        print(f"Error: {e}")
        continue


  0%|          | 0/127 [00:00<?, ?it/s]

Processing Buenos Aires
Segmentation shape after filter:  2544226
Segmentation shape after filter images with problems:  2543977
data consistent
Segmentation shape:  2543977
Variables original:  31
Variables kept:  30


  1%|          | 1/127 [00:51<1:47:22, 51.13s/it]

city buenosaires saved
Processing Sydney
Segmentation shape after filter:  3246927
Segmentation shape after filter images with problems:  3246647
data consistent
Segmentation shape:  3155091
Variables original:  31
Variables kept:  30


  2%|▏         | 2/127 [02:49<3:09:20, 90.88s/it]

...

city houston saved
Processing Philadelphia
Segmentation shape after filter:  5177748
Segmentation shape after filter images with problems:  5177328
data consistent
Segmentation shape:  4964552
Variables original:  31
Variables kept:  30


 99%|█████████▉| 126/127 [2:23:39<01:34, 94.60s/it]

city philadelphia saved
Processing Chicago
Segmentation shape after filter:  1632887
Segmentation shape after filter images with problems:  1632603
data consistent
Segmentation shape:  1159541
Variables original:  31
Variables kept:  30


100%|██████████| 127/127 [2:25:44<00:00, 68.85s/it] 

city chicago saved


In [2]:
# check the exported data
cityabbr = "hongkong"
seg_df_summary_update = pd.read_csv(f"{EXPORT_FOLDER}/{cityabbr}_seg_summary.csv")

In [2]:
# load seg summary for all cities and concat
seg_df_all_h3 = []
city_ls = pd.read_csv("../city_meta.csv")["City"].unique().tolist()
for city in tqdm(city_ls):
    cityabbr = city.lower().replace(" ", "")
    temp = pd.read_csv(f"{EXPORT_FOLDER}/{cityabbr}_seg_summary.csv")
    temp['city_lower'] = cityabbr
    seg_df_all_h3.append(temp)
seg_df_all_h3 = pd.concat(seg_df_all_h3).reset_index(drop = True)
seg_df_all_h3.head()

  0%|          | 0/127 [00:00<?, ?it/s]

100%|██████████| 127/127 [00:08<00:00, 15.49it/s]


,h3_9,sidewalk_indicator,exposure_indicator,img,panoid,sidewalk_presence,exposure_presence,obj_bench,obj_bicycle,obj_bus,obj_car,obj_chair,obj_fire hydrant,obj_motorcycle,obj_person,obj_stop sign,obj_traffic light,obj_train,obj_truck,city_lower
0,89c2e302003ffff,87,92,92,23,0.945652,1.0,0.000000,0.142857,1.309524,25.404762,0.0,0.071429,0.428571,2.309524,0.000000,1.476190,0.000000,1.904762,buenosaires
1,89c2e302007ffff,107,108,108,27,0.990741,1.0,0.013289,0.172757,0.833887,26.524917,0.0,0.016611,0.750831,3.392027,0.000000,0.674419,0.006645,0.973422,buenosaires
2,89c2e30200bffff,109,112,112,28,0.973214,1.0,0.000000,0.018349,0.633028,21.064220,0.0,0.009174,0.137615,2.559633,0.055046,0.614679,0.018349,0.853211,buenosaires
3,89c2e30200fffff,118,120,120,30,0.983333,1.0,0.000000,0.000000,0.418182,26.490909,0.0,0.018182,0.163636,1.836364,0.000000,0.000000,0.000000,0.872727,buenosaires
4,89c2e302013ffff,106,108,108,27,0.981481,1.0,0.029340,0.112469,0.687042,24.848411,0.0,0.019560,0.616137,2.948655,0.009780,0.787286,0.017115,1.178484,buenosaires


In [5]:
seg_df_all_h3.to_csv(os.path.join(EXPORT_FOLDER, 'c_exposure_sidewalk_h3.csv'), index = False)

In [4]:
seg_df_all_h3.groupby(['city_lower'])['exposure_presence'].mean()

city_lower
accra         0.978613
amsterdam     0.834003
antwerp       0.906148
astrakhan     0.915536
athens        0.771027
                ...   
vijayawada    0.855779
warsaw        0.843687
wellington    0.653854
yamaguchi     0.668904
zwolle        0.829262
Name: exposure_presence, Length: 127, dtype: float64